In [ ]:
import holoviews as hv
import geoviews as gv
import panel as pn
import pandas as pd
import geopandas as gpd
import fiona
import re
import subprocess

hv.extension('bokeh')
gv.extension('bokeh')
pn.extension()

fiona.drvsupport.supported_drivers["KML"] = "rw"  # Enable geopandas KML i/o support

# Hickman's dataset
These cells show the visualization of Josh Hickman's dataset of the Android 13 device.
The dataset is provided as a reference so this should just run and give you an idea how to visualize your own dataset.
After plotting, you can pan and zoom with your mouse and scroll-wheel to look around.

In [ ]:
# Load data
tiles_df = gpd.GeoDataFrame.from_file('../testdata/hickman_13.geojson')
kml = gpd.read_file("../testdata/hickman_route.kml", driver='KML')

# Prepare visualization
basemap = gv.tile_sources.CartoLight  # Lighter than OSM
# basemap = gv.tile_sources.OSM
tiles = gv.Contours(tiles_df['geometry'].tolist()).opts(color='black')
kml_route = gv.Path(kml.iloc[0]['geometry']).opts(color='blue', line_width=5)

(basemap * kml_route * tiles).opts(width=600, height=600, active_tools=['wheel_zoom','pan'])

# Experiment visualization
These cells allow for the loading and visualization of data acquired from a reference experiment.
First, record an experiment using an AVD or physical rooted device using `experiment.py`.
Then decrypt the collected `map_cache.db` with its corresponding `map_cache.key` using the script: `decrypt_map_cache.py`.
The output of this decryption should be placed in: `/tmp/experiment.geosjon` for this example.

In [ ]:
# Load the output of decrypt_map_cache.py in a dataframe
tiles = gpd.GeoDataFrame.from_file('/tmp/experiment.geojson')
tiles['timestamp'] = tiles['timestamp'].dt.tz_localize('Europe/Amsterdam')
# tiles

In [ ]:
# Load the recorded locations from the experiment in a dataframe
experiment_locations = pd.read_csv('/tmp/experiment_locations.csv', parse_dates=['timestamp'])
experiment_locations['timestamp'] = experiment_locations['timestamp'].dt.tz_convert('Europe/Amsterdam')

In [ ]:
# Calculate the video start timestamp based on exifdata and its duration

def get_video_start(filepath):
    exif = subprocess.getoutput(f'exiftool {filepath}')
    # Create Date seems to match the time the video ends and is thus written to storage?
    create_date = re.findall(r'^\s*Create Date\s+: (.*)', exif, re.MULTILINE)[0]
    create_date = pd.to_datetime(create_date, format='%Y:%m:%d %H:%M:%S').tz_localize('UTC')
    duration = re.findall(r'^Duration\s+: (.*)', exif, re.MULTILINE)[0]
    duration = pd.to_timedelta(duration)
    return (create_date - duration).tz_convert('Europe/Amsterdam')

vid_start = get_video_start('/tmp/experiment_video.mp4')
# Show timestamps to visually confirm they are aligned in the same timezone
# vid_start, tiles['timestamp'], experiment_locations['timestamp']

In [ ]:
# Visualize the experiment by synchronizing the screenrecording with the tile data on an interative map

def get_plot(t):
    t = (vid_start + pd.Timedelta(seconds=t))  # Convert elapsed video time (seconds) to timestamp
    location = experiment_locations.iloc[(experiment_locations['timestamp'] - t).abs().argsort()[:1]]  # Get experiment locations nearest to timestamp

    old_tiles = tiles[(tiles['timestamp'] < t)]
    cache_hit_tiles = tiles[(tiles['timestamp'] - t).abs() < pd.Timedelta(seconds=1)]
    return (gv.Contours(old_tiles['geometry'].to_list()).opts(color='lightgray') * \
            gv.Contours(cache_hit_tiles['geometry'].to_list()).opts(color='blue') * \
            gv.Points(location, kdims=['lon', 'lat'], ).opts(marker='o', size=10, color='b'))


SIZE=600
vid = pn.pane.Video('/tmp/experiment_video.mp4', height=SIZE)
dmap = (gv.tile_sources.CartoLight * hv.DynamicMap(pn.bind(get_plot, vid.param.time))).opts(height=SIZE, width=SIZE)
pn.Row(vid, dmap)